# 02 - Calibration du modèle de redressement

Ce notebook calibre le modèle de correction brut → net (redressement des sondages).

## Méthodologie

1. **Données de calibration** : 4 élections historiques à Paris
   - Municipales 2014 & 2020
   - Présidentielle 2022 (T1)
   - Européennes 2024

2. **Méthodes** :
   - Multiplicative : ratio résultat/sondage
   - Additive : différence résultat - sondage

3. **Pondération temporelle** : demi-vie de 2 élections (récence)

4. **Validation** : leave-one-out cross-validation

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display

from paris_elections.redressement.model import (
    RedressementModel,
    CorrectionMethod,
)
from paris_elections.redressement.calibration import (
    build_calibration_points,
    build_model,
    leave_one_out_validation,
    overall_mae,
    ALL_CALIBRATION,
)
from paris_elections.config import POLITICAL_FAMILIES

## 1. Données de calibration

In [ ]:
# Afficher les données intégrées
calibration_df = []
for election, (year, data) in ALL_CALIBRATION.items():
    for family, poll, actual in data:
        calibration_df.append({
            'election': election,
            'year': year,
            'family': family,
            'poll_score': poll,
            'actual_score': actual,
            'delta': actual - poll,
            'ratio': actual / poll if poll > 0 else 1.0,
        })

calibration_df = pd.DataFrame(calibration_df)
display(calibration_df)

In [ ]:
# Statistiques par famille politique
stats = calibration_df.groupby('family').agg({
    'delta': ['mean', 'std', 'count'],
    'ratio': ['mean', 'std'],
}).round(2)
display(stats)

## 2. Visualisation : sondage vs résultat

In [ ]:
fig = px.scatter(
    calibration_df,
    x='poll_score',
    y='actual_score',
    color='family',
    symbol='election',
    title='Sondage vs Résultat — Paris (2014-2024)',
    labels={'poll_score': 'Score sondage (%)', 'actual_score': 'Score réel (%)'},
    hover_data=['election', 'year'],
)

# Ligne y=x (sondage parfait)
fig.add_trace(go.Scatter(
    x=[0, 40], y=[0, 40],
    mode='lines',
    line=dict(dash='dash', color='gray'),
    name='Parfait',
))

fig.update_layout(template='plotly_white')
fig.show()

## 3. Construction du modèle multiplicatif

In [ ]:
model_mult = build_model(method=CorrectionMethod.MULTIPLICATIVE, half_life=2.0)

print("Facteurs de correction (multiplicatif) :")
print("="*50)
summary = model_mult.summary()
for family, info in sorted(summary.items()):
    factor = info['factor']
    std = info['std']
    n = info['n_points']
    ci = info['ci_95']
    interpretation = "surestimé" if factor < 1 else "sous-estimé" if factor > 1 else "juste"
    print(f"  {family:5} : ×{factor:.2f} (±{std:.2f}) [{n} pts] → {interpretation}")

## 4. Construction du modèle additif

In [ ]:
model_add = build_model(method=CorrectionMethod.ADDITIVE, half_life=2.0)

print("Facteurs de correction (additif) :")
print("="*50)
summary = model_add.summary()
for family, info in sorted(summary.items()):
    factor = info['factor']
    std = info['std']
    sign = "+" if factor > 0 else ""
    print(f"  {family:5} : {sign}{factor:.1f} pts (±{std:.1f})")

## 5. Validation croisée leave-one-out

In [ ]:
# Méthode multiplicative
loo_mult = leave_one_out_validation(CorrectionMethod.MULTIPLICATIVE)
mae_mult = overall_mae(CorrectionMethod.MULTIPLICATIVE)

print(f"MAE globale (multiplicatif) : {mae_mult:.2f} points")
print("\nErreurs par élection :")
for election, errors in loo_mult.items():
    avg = np.mean(list(errors.values()))
    print(f"  {election}: MAE = {avg:.2f} pts")

In [ ]:
# Méthode additive
loo_add = leave_one_out_validation(CorrectionMethod.ADDITIVE)
mae_add = overall_mae(CorrectionMethod.ADDITIVE)

print(f"MAE globale (additif) : {mae_add:.2f} points")
print("\nErreurs par élection :")
for election, errors in loo_add.items():
    avg = np.mean(list(errors.values()))
    print(f"  {election}: MAE = {avg:.2f} pts")

## 6. Application du redressement

In [ ]:
# Exemple : sondage fictif
sondage_brut = {
    'PS': 18.0,
    'LFI': 15.0,
    'EELV': 8.0,
    'REN': 20.0,
    'LR': 16.0,
    'RN': 12.0,
    'REC': 6.0,
    'DIV': 5.0,
}

# Appliquer le redressement
sondage_redresse = model_mult.correct(sondage_brut)

print("Redressement d'un sondage fictif :")
print("="*50)
print(f"{'Liste':<8} {'Brut':>8} {'Redressé':>10} {'Δ':>8}")
print("-"*50)
for liste in sondage_brut:
    brut = sondage_brut[liste]
    redresse = sondage_redresse.get(liste, brut)
    delta = redresse - brut
    sign = '+' if delta > 0 else ''
    print(f"{liste:<8} {brut:>7.1f}% {redresse:>9.1f}% {sign}{delta:>7.1f}")

## 7. Bandes d'incertitude

In [ ]:
bands = model_mult.uncertainty_band(sondage_brut)

bands_df = pd.DataFrame([
    {'liste': k, 'low': v[0], 'central': v[1], 'high': v[2]}
    for k, v in bands.items()
])

fig = go.Figure()

fig.add_trace(go.Bar(
    x=bands_df['liste'],
    y=bands_df['central'],
    error_y=dict(
        type='data',
        symmetric=False,
        array=bands_df['high'] - bands_df['central'],
        arrayminus=bands_df['central'] - bands_df['low'],
    ),
    marker_color=[POLITICAL_FAMILIES.get(l, POLITICAL_FAMILIES['DIV']).color 
                  for l in bands_df['liste']],
))

fig.update_layout(
    title='Scores redressés avec IC 95%',
    xaxis_title='Liste',
    yaxis_title='Score (%)',
    template='plotly_white',
)
fig.show()

## 8. Conclusion

### Observations clés :

1. **RN** : fortement surestimé par les sondages à Paris (×0.5-0.7)
2. **REN** : légèrement sous-estimé (×1.05-1.15)
3. **PS** : variable, dépend du contexte
4. **LFI** : tendance à la surestimation légère

### Limites :

- Peu de points de calibration (4 élections)
- Contextes électoraux différents (municipales ≠ présidentielle)
- **→ Incertitude élevée, bien communiquée via Monte Carlo**